### Importação de Módulos

In [1]:
# Builtins
import logging
import os
import random
import threading
from datetime import date, datetime

# Third party
import pandas as pd
import requests
import json
import time

In [2]:
version = 'HOMOLOG'
keyversion = 'HOM'

### Configuração de Logs

In [3]:
logger = logging.getLogger(f"{version}")
today = str(date.today()) + '-' + str(int(time.time()))
logging.basicConfig(filename=f'/work/execution_logs/{today}.txt', encoding='utf-8', level=logging.INFO)

## Classes

### API de mudança de Planos

In [4]:
class AstreaPlanApi:

    def __init__(self, auth):
        """
        Inicializa a API de mudança de planos.
        """
        self.headers = {
            'Authorization': auth,
            'Content-Type': 'application/json'
            }
        if version == 'HOMOLOG':
            self.url = 'https://change-plan-dot-astreav2.appspot.com/api/v2/dev/change-plan'
        elif version == 'PRODUCAO':
            self.url = 'https://app.astrea.net.br/api/v2/dev/change-plan'
        else:
            raise Exception(f'Versão de API inválida: {version}')
            
        self.url = 'https://change-plan-dot-astreav2.appspot.com/api/v2/dev/change-plan'
        self.semaphore = threading.Semaphore(3) # Limit the number of concurrent requests
        self.request_queue = [] # Queue for requests that exceed the concurrent limit
        self.successful_requests = 0 # Track number of successful requests
        self.error_count = 0 # Track number of errors
        logger.info(f"{__class__.__name__}: API de mudança de planos inicializada")

    def get_payload(self, customer):
        """
        Cria o payload para a requisição de mudança de plano

        Params:
        - Customer: Dicionário contendo a informação do cliente a ser integrado

        Retorna:
        - Dicionário com o payload formatado para a api do Astrea.
        """
        payload = json.dumps({
            "tenantId": customer['api_key'],
            "editionType": customer['plano_destino'],
            "baseValue": customer['vr_bruto_destino'],
            "discount": customer['desconto_destino']
        })
        logger.info(f'{__class__.__name__}: Payload obtido para o customer: {customer["namespace"]} :' + str(payload))
        return payload

    def make_change_plan_request(self, customer):
        """
        Realiza uma requisição POST para a API de mudança de planos.

        Params:
        - Customer: Dicionário contendo a informação do cliente a ser integrado

        Retorna:
        - O objeto Response da requisição.
        """
        payload = self.get_payload(customer)
        
        response = requests.post(self.url, headers= self.headers, data=payload)
            
        if response.status_code in (200, 201):
            logger.info(f'{__class__.__name__}: Resposta para customer: {customer["namespace"]}: ' + str(response.status_code) + ' - ' + str(response.content) + f' ({response.elapsed.total_seconds()}s)')
            self.successful_requests += 1
        else:
            self.error_count += 1
            logger.info(f'{__class__.__name__}: Resposta para customer: {customer["namespace"]}: ' + str(response.status_code) + ' - ' + str(response.content) + f' ({response.elapsed.total_seconds()}s)')
            
        return response 

### Api do Intercom

In [5]:
class IntercomAPI:
    def __init__(self, auth):
        self.headers = {
            "accept": "application/json",
            "content-type": "application/json",
            "authorization": auth
        }
        logger.info(f"{__class__.__name__}: API do Intercom Inicializada")


    def get_company_by_name(self, company_name):
        url = f"https://api.intercom.io/companies?name={company_name}"
        response = requests.get(url, headers=self.headers)
        if response.ok:
            logger.info(f"{__class__.__name__}: Company {company_name} encontrada.")
        else:
            logger.info(f"{__class__.__name__}: Company {company_name} não encontrada.")
        return response.json()
    
    def get_contacts_by_company_id(self, company_id):
        url = f"https://api.intercom.io/companies/{company_id}/contacts"
        response = requests.get(url, headers=self.headers)
        if response.ok:
                logger.info(f"{__class__.__name__}: {len(response.json()['data'])} contatos encontrados para a company {company_id}.")
        else:
            logger.info(f"{__class__.__name__}: A busca de contatos falhou para a company {company_id}.")
        return response.json()["data"]
    
    def send_event(self, company_id, user, event_name, timestamp):
        url = "https://api.intercom.io/events"
        payload = {
            "user_id": user['email'],
            "created_at": timestamp,
            "event_name": f"{event_name}",
            "id": user['id']
        }
        response = requests.post(url, json=payload, headers=self.headers)
        if response.ok:
            logger.info(f"{__class__.__name__}: Evento {event_name} enviado para o usuário {user['email']}.")
        else:
            logger.info(f"{__class__.__name__}: Evento {event_name} não enviado para o usuário {user['email']}. Código de erro: {response.status_code}")
        return response

    def notify_company_users(self, company_name, event_name, timestamp):
        company = self.get_company_by_name(company_name)
        users = self.get_contacts_by_company_id(company['id'])
        errors = 0
        for user in users:
            response = self.send_event(company['id'], user, event_name, timestamp)
            if response.ok == False:
                errors += 1
        logger.info(f"{__class__.__name__}: {len(users)} usuários na company {company_name} foram notificados com {errors} erros.")
        

## Funções Personalizadas

### Notificações de Execução

In [6]:
# Slack Notify: Envia um webhook para um zap que realiza a integração de mensagens para o slack
def slack_notify(service,status,message):
    url = f'https://hooks.zapier.com/hooks/catch/52679/3d690x6/?service={service}&status={status}&message={message}'
    response = requests.request("GET", url)
    logger.info(f"{__name__}: {service}'s {status} message sent to zapier: {message}")
    return True

## Extrações de Dados

In [7]:
slack_notify(f'Virada de Planos - {version}','Start','Iniciando Extração de dados para Virada de Planos')

True

### Consulta de clientes no astrea_data

In [8]:
df_clientes = _deepnote_execute_sql('select vsn.api_key,\n       vcc.namespace,\n       vsn.periodicidade_atual as periodicidade_origem,\n       vcc.plano as plano_origem,\n\n       case\n        when vsn.minimo_plano_aumento < 0 then \'VIP\'\n        when vsn.minimo_plano_aumento >= 0 then \'COMPANY\'\n        else Null\n       end as plano_destino,\n\n       vcc.vr_contrato_total_com_cupons as vr_liquido_origem,\n\n       vcc.vr_contrato_total_sem_cupons as vr_bruto_origem,\n\n       case\n        when vsn.minimo_plano_aumento < 0 then vsn.prox_plano_contrato_sem_cupons\n        when vsn.minimo_plano_aumento >= 0 then np.vr_contrato_total_sem_cupons\n        else Null\n       end as vr_bruto_destino,\n\n       vcc.vr_cupons_ativos_total as desconto_origem,\n\n       case\n        when vsn.minimo_plano_aumento < 0 then vsn.prox_plano_contrato_sem_cupons - vcc.vr_contrato_total_com_cupons\n        when vsn.minimo_plano_aumento >= 0 then 0\n        else Null\n       end as desconto_destino,\n\n       vcc.parcela_dt_proxima,\n\n       vcc.parcela_dt_proxima::date - current_date as dias_ate_renovacao\n\nfrom cx_data.v_simulacao_novos_planos vsn\njoin cx_data.novos_planos np on np.id = vsn.minimo_plano_id\njoin oroboro2.v_contratos_consolidado vcc on vcc.api_key = vsn.api_key\nleft join oroboro2.contratos_custom cc on vcc.id = cc.contrato_id\nwhere vsn.minimo_plano_destino = \'Company\'', 'SQL_EB56FC30_B625_4AA0_83D2_B8F22F562A50', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_clientes

,api_key,namespace,periodicidade_origem,plano_origem,plano_destino,vr_liquido_origem,vr_bruto_origem,vr_bruto_destino,desconto_origem,desconto_destino,parcela_dt_proxima,dias_ate_renovacao
0,5633421678215168,Soares_e_Souza,Mensal,Astrea Pro 217 / 50,COMPANY,443.40,443.40,557.0,0.00,0.0,2023-09-10 00:00:00+00:00,19
1,5721310499962880,Claro_e_DC_Fonseca,Mensal,Astrea Pro 237 / 60,COMPANY,391.81,391.81,557.0,0.00,0.0,2023-09-26 00:00:00+00:00,35
2,5665395394478080,Laurindo_Freitas_e_Selva,Mensal,Astrea Starter Padrão 50 / 25,VIP,561.60,561.60,997.0,0.00,435.4,2023-09-10 00:00:00+00:00,19
3,5180660218068992,Aguiar_Toledo_Advogados_Associados,Mensal,Astrea Starter Padrão 50 / 25,COMPANY,185.44,185.44,557.0,0.00,0.0,2023-09-10 00:00:00+00:00,19
4,5390824133099520,giovannidilion,Mensal,Astrea Starter Padrão 60 / 25,COMPANY,385.33,469.25,557.0,83.92,0.0,2023-09-20 00:00:00+00:00,29
...,...,...,...,...,...,...,...,...,...,...,...,...
2502,5834965021753344,contato118874,Mensal,Astrea Pro 277 / 70,COMPANY,420.00,577.00,557.0,157.00,0.0,2023-08-23 00:00:00+00:00,1
2503,5115392389677056,ollier.adv,Anual,Astrea Pro 277 / 70,COMPANY,4500.00,4661.40,5844.0,161.40,0.0,2024-08-24 00:00:00+00:00,368
2504,5172001582219264,otavioalday,Mensal,Astrea Pro 277 / 70,COMPANY,497.00,497.00,557.0,0.00,0.0,2023-08-23 00:00:00+00:00,1
2505,6098603280302080,joaovitor24,Anual parcelado,Astrea Pro 277 / 70,COMPANY,4000.00,4406.40,5844.0,406.40,0.0,2023-08-23 00:00:00+00:00,1


## Fluxo de Controle

In [9]:
control_flow_begin = int(time.time())
slack_notify(f'Virada de Planos - {version}','Info','Base de Clientes extraída, filtrando clientes para atualização')

True

In [10]:
df_clientes_execucao = _deepnote_execute_sql('select * from df_clientes\nwhere  (periodicidade_origem like \'%Anual%\' \n    or (periodicidade_origem like \'%Mensal%\'\n        and parcela_dt_proxima::date >= \'2023-10-05\'::date))\nlimit 3', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
df_clientes_execucao

,api_key,namespace,periodicidade_origem,plano_origem,plano_destino,vr_liquido_origem,vr_bruto_origem,vr_bruto_destino,desconto_origem,desconto_destino,parcela_dt_proxima,dias_ate_renovacao
0,4721891575070720,heitor.amaral60,Anual,Astrea Pro 277 / 70,COMPANY,3378.52,3378.52,5844.0,0.0,0.0,2023-09-20,29
1,5131375925526528,advmarinapacheco,Anual parcelado,Astrea Starter Padrão 60 / 25,COMPANY,1561.80,1561.80,5844.0,0.0,0.0,2024-07-03,316
2,6453087600377856,contato84,Anual,Astrea Starter Padrão 60 / 25,COMPANY,2725.87,2725.87,5844.0,0.0,0.0,2023-10-10,49


In [11]:
# Instanciando API do Astrea
apiastrea = AstreaPlanApi(auth = os.environ[f"ASTREAV2_{keyversion}_AUTH"])

# Instanciando a API do Intercom
apiintercom = IntercomAPI(auth = os.environ["INTERCOM_AUTH"])

In [12]:
clientes_exec = len(df_clientes_execucao)

if clientes_exec == 0:
    slack_notify(f'Virada de Planos - {version}','Info','Nenhum cliente encontrado para atualização')

elif clientes_exec > 0:
    slack_notify(f'Virada de Planos - {version}','Info',f'{clientes_exec} clientes encontrados para atualização')
    logger.info('Fluxo de Controle: Iniciando o Loop de integração')
    for index, row in df_clientes_execucao.iterrows():
        customer = row.to_dict()
        request_at = int(time.time())
        logger.info(f'{__name__}: Integrando customer: {customer["namespace"]} em {request_at}')
        response = apiastrea.make_change_plan_request(customer=customer)
        response_at = int(time.time())

        if response.status_code in (200, 201):
            apiintercom.notify_company_users(
                company_name=row["namespace"],
                event_name="virada-planos-automacao-concluida",
                timestamp=response_at
            )
        else:
            apiintercom.notify_company_users(
                company_name='jackeline.chagas',
                event_name="virada-planos-automacao-falhou",
                timestamp=response_at
            )

control_flow_end = int(time.time())
total_elapsed_time = control_flow_end - control_flow_begin
logger.info(f'Atualização finalizada ({total_elapsed_time}s), {apiastrea.error_count} erros, {apiastrea.successful_requests} sucessos.')
slack_notify(f'Virada de Planos - {version}','Finish',f'Atualização finalizada ({total_elapsed_time}s), {apiastrea.error_count} erros, {apiastrea.successful_requests} sucessos.')
logger.info(f"Fluxo de Controle Finalizado")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a6aa03ff-9a9e-4b1a-a1ec-ff555e98a018' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>